<a href="https://colab.research.google.com/github/KaabiTaoufik/ImageProcessing/blob/main/ImageProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [245]:
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from PIL import Image
import matplotlib.image
import random
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [246]:
imageFilePath = '/content/drive/MyDrive/imageProcessing/chat.pgm'
def read_pgm(pgmf):
   header = pgmf.readline()
   fileType = header.split()[0]
   header = pgmf.readline()
   header = pgmf.readline()
   (width, height) = [int(i) for i in header.split()[0:2]]
   header = pgmf.readline()
   maxVal = int(header.split()[0])
   matrix = []
   for y in range(height):
       row = [int(i) for i in pgmf.readline().split()]
       matrix.append(row)
   return (matrix, height, width, maxVal, fileType)

f = open(imageFilePath, 'rb')
im = read_pgm(f)
f.close()
(im, lx, ly , maxVal, fileType) = np.array(im)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [247]:
mean = np.mean(im)
print("mean : ", mean)

mean :  85.43944010416666


In [248]:
a = im
hist, bins = np.histogram(a, bins=maxVal)
print()
print (hist)
print()

# Creating plot  #uncomment to show plot ( very slow due to size being 255)
#fig = plt.figure(figsize =(10, 7))
#plt.hist(a, bins = maxVal)
#plt.title("chat histogram")
#plt.show()


[2026 3623 2663 2783 1735  848 1326 1794  765  490  412  369  362  297
  305  247  280  289  281  266  271  265  281  298  272  292  299  321
  277  315  300  305  329  316  327  326  399  352  356  374  387  377
  382  406  366  376  407  360  343  389  352  424  374  316  337  357
  337  343  321  261  316  284  332  244  291  276  264  288  252  257
  251  264  230  202  227  237  227  240  231  230  203  222  222  225
  220  220  235  197  224  212  211  223  241  231  222  230  237  211
  263  223  254  253  248  269  264  249  260  245  227  256  249  264
  248  273  257  283  281  275  269  301  277  275  256  286  284  275
  291  307  312  296  330  279  320  324  298  334  320  345  337  323
  304  310  341  323  305  324  304  303  316  337  287  330  297  323
  333  366  298  353  370  330  328  356  338  316  358  369  297  344
  281  302  309  303  320  323  280  316  247  255  289  268  253  252
  255  210  246  234  255  234  210  223  199  193  201  156  174  154
  153

In [249]:
def writePGM(lx, ly, maxVal, matrix):
  # define the width  (columns) and height (rows) of your image
  height = lx
  width = ly
  p_num = width * height

  # open file for writing 
  filename = '/content/drive/MyDrive/imageProcessing/test.pgm'
  fout=open(filename, 'wb')

  # define PGM Header
  pgmHeader = 'P2' + '\n'

  pgmHeader_byte = bytearray(pgmHeader,'utf-16')

  # write the header to the file
  fout.write(pgmHeader_byte)

  pgmComment = '# ' + filename + ' ' + 'nbre de lignes :' + str(height) + ' nbre de collones :' + str(width) +'\n'
  pgmComment_byte = bytearray(pgmComment,'utf-16')
  fout.write(pgmComment_byte)
  # write the data to the file 

  heightAndWidth = str(width) + ' ' + str(height) + '\n'
  heightAndWidth_byte = bytearray(heightAndWidth,'utf-16')
  fout.write(heightAndWidth_byte)

  maxVal = str(maxVal) + '\n'
  maxVal_byte = bytearray(maxVal,'utf-16')
  fout.write(maxVal_byte)

  for j in range(height):
      bnd = ' '.join(map(str,list(im[j]))) + '\n'
      fout.write(bytearray(bnd,'utf-16'))

  fout.close()

In [250]:
def saveImageAsPNG(matrix, imageName):
  matplotlib.image.imsave('/content/drive/MyDrive/imageProcessing/'+ str(imageName) + '.png', matrix)

In [251]:
def addNoise(matrix, lx, ly):
  mat = np.array(matrix)
  for i in range(lx):
    for j in range(ly):
      randomNb = random.randint(0,10)
      if randomNb == 0:
        mat[i][j] = 0
      if randomNb == 10:
        mat[i][j] = 255
  return mat

In [252]:
NoisyImg = addNoise(im, lx, ly)
saveImageAsPNG(NoisyImg, 'CatWithNoise')

In [259]:
def calculateMeanInRange(filter, mat, i, j, n):
  mean = 0
  for k in range(n):
    for c in range(n):
      mean += mat[i-k][j-c]*filter[k][c]
  mean = mean / (n*n)
  return mean

In [260]:
def linearTransform(filter, mat, lx, ly):
  (n,n) = filter.shape
  newMat = np.array(mat)
  for i in range(lx):
    for j in range(ly):
      if( not(i - n < 0 
         or i + n > lx
         or j - n < 0
         or j + n > ly)):
        newMat[i][j] = calculateMeanInRange(filter, mat, i, j, n)
      else:
        newMat[i][j] = 255
  return newMat

In [261]:
filter = np.ones((3,3))
linearTransformed = linearTransform(filter, NoisyImg, lx, ly)
saveImageAsPNG(linearTransformed, "linearTransform")

In [256]:
def calculateMedianInRange(mat, i, j, n):
  List = []
  for k in range(n):
    for c in range(n):
      List.append(mat[i-k][j-c])
  sortedList = np.sort(List)
  return sortedList[int((n*n)/2)]

In [257]:
def medianTransform(size, mat , lx ,ly):
  n = size
  newMat = np.array(mat)
  for i in range(lx):
    for j in range(ly):
      if( not(i - n < 0 
         or i + n > lx
         or j - n < 0
         or j + n > ly)):
        newMat[i][j] = calculateMedianInRange(mat, i, j, n)
      else:
        newMat[i][j] = 255
  return newMat

In [258]:
size = 3
medianTransform = medianTransform(size, NoisyImg, lx, ly)
saveImageAsPNG(medianTransform, "MedianTransform")

In [263]:
#image enhancing
filter = np.array([
    [1 , 0 , -1],
    [2 , 0 , -2],
    [1 , 0 , -1]
])
enhancedTransform = linearTransform(filter, im, lx, ly)
saveImageAsPNG(enhancedTransform, "enhancedTransform")

In [264]:
def coVar(mat, mean, lx, ly):
  coVar = 0
  for i in range(lx):
    for j in range(ly):
      coVar += pow((mat[i][j] - mean),2)
  return coVar

In [269]:
def SignalNoiseRatio(originalMat, FilteredMat, mean, lx, ly):
  S = coVar(originalMat, mean, lx, ly)
  N = coVar(FilteredMat, mean, lx, ly)
  return S/N

In [270]:
print("S/B for linearTransformation",SignalNoiseRatio(im,linearTransformed, mean, lx, ly))
print("S/B for medianTransformion",SignalNoiseRatio(im,medianTransform, mean, lx, ly))


S/B for linearTransformation 1.0848140983920997
S/B for medianTransformion 0.861970852756851
